In [1]:
!pip install fastai lime shap torchcam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=6dfe56e5be7fe4c84b2e6f588fdfe466484eae9479225fa33d1286158721c5ac
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch

# CUDA 가용 여부 확인
if torch.cuda.is_available():
    print("CUDA is available. PyTorch is using GPU.")
    print(f"Device: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. PyTorch is using CPU.")

print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

CUDA is available. PyTorch is using GPU.
Device: Tesla T4
0
Tesla T4


In [4]:
# 경로 설정
drive_dir = "drive/My Drive/mldl/"
dataset_dir = drive_dir + "dataset/"
data_dir = drive_dir + "dataset/data/"
model_dir = "model/"
test_dir = drive_dir + "dataset/test/"
grad_dir = drive_dir + "dataset/gradcam/"

In [5]:
from fastai.vision.all import *
from fastai.vision.augment import RandomResizedCrop

# 경로 설정
path = Path(data_dir)

def safe_open_image(fn):
    try:
        with Image.open(fn) as img:
            img.verify()
            img.load()
        return PILImage.create(fn)
    except Exception as e:
        print(f"이미지 열기 실패 {fn}: {e}")
        return None

def get_items(path):
    return [o for o in get_image_files(path) if safe_open_image(o) is not None]

# 데이터 로드 (훈련과 검증 세트)
dls = ImageDataLoaders.from_folder(
    path,
    train='train',
    valid_pct=0.2,
    get_items=get_items,
    item_tfms=Resize(460),
    batch_tfms=[*aug_transforms(size=224), Normalize.from_stats(*imagenet_stats)],
    num_workers=0  # 멀티프로세싱 비활성화
)

In [6]:
print(dls.vocab)

['0_real', '1_fake']


In [7]:
import shap
import lime
from lime import lime_image
import numpy as np
import matplotlib.pyplot as plt

# 모델 불러오기
learn = vision_learner(dls, resnet34, metrics=accuracy)
learn.model_dir = model_dir
learn.load('Resnet34_v1')

# 테스트 이미지 경로 설정
test_image_path = Path(test_dir)

# 디렉토리 내의 모든 이미지에 대해 예측 수행 함수
def predict_images_in_directory(directory_path):
    # 디렉토리 내의 모든 이미지 파일 가져오기
    image_files = get_image_files(directory_path)

    # 각 이미지에 대해 예측
    for image_file in image_files:
        img = PILImage.create(image_file)
        pred_class, pred_idx, probs = learn.predict(img)
        print(f"이미지: {image_file.name}, 예측된 클래스: {pred_class}, 확률: {probs[pred_idx]:.4f}")

# 테스트 디렉토리 내 모든 이미지에 대해 예측 수행
predict_images_in_directory(test_image_path)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 113MB/s]
/usr/local/lib/python3.10/dist-packages/fastai/learner.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have

이미지: 0479.jpg, 예측된 클래스: 0_real, 확률: 0.9073


이미지: 0490.jpg, 예측된 클래스: 1_fake, 확률: 0.8943


이미지: 0477.jpg, 예측된 클래스: 0_real, 확률: 0.9998


이미지: 0483.jpg, 예측된 클래스: 0_real, 확률: 0.9675


...

이미지: zzyzicbnnv.png, 예측된 클래스: 0_real, 확률: 1.0000


이미지: zzzkuyzysa.png, 예측된 클래스: 1_fake, 확률: 0.8994


이미지: zzybuwyuro.png, 예측된 클래스: 1_fake, 확률: 0.9995


이미지: zzykiwwjgm.png, 예측된 클래스: 1_fake, 확률: 0.9999


이미지: zzyxaefoft.png, 예측된 클래스: 1_fake, 확률: 0.9998


In [8]:
from fastai.vision.all import *
import matplotlib.pyplot as plt
import cv2
from IPython.display import clear_output

def apply_gradcam(learn, img_path):
    # 이미지 로드 및 전처리
    img = PILImage.create(img_path)

    # 원본 이미지와 네트워크에 들어가는 이미지 크기를 일치시키기 위해서 transforms 사용
    dl = learn.dls.test_dl([img_path])
    batch = dl.one_batch()
    x = batch[0] if isinstance(batch, (tuple, list)) else batch
    x = x.to(next(learn.model.parameters()).device)

    # Grad-CAM 결과 계산
    cam = gradcam(learn, x)

    # 원본 이미지와 GradCAM 결과를 표시할 플롯 생성
    # fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # 원본 이미지 표시
    # ax1.imshow(img)
    # ax1.set_title('Original Image')
    # ax1.axis('off')

    # GradCAM 결과는 224x224 크기의 활성화 맵이므로 원본 이미지 크기에 맞게 리사이즈
    cam_resized = cv2.resize(cam, (img.size[0], img.size[1]))

    # GradCAM 결과 표시 (원본 이미지에 오버레이)
    # ax2.imshow(img)
    # ax2.imshow(cam_resized, alpha=0.6, cmap='jet')
    # ax2.set_title('GradCAM')
    # ax2.axis('off')

    # 예측 결과 계산
    pred_class, pred_idx, probs = learn.predict(img)
    pred_info = f"Predicted Class: {pred_class}\nProbability: {probs[pred_idx]:.4f}"

    # 이미지 아래에 예측 결과 텍스트 추가
    # fig.text(0.5, -0.05, pred_info, ha='center', va='top', fontsize=12, color='black', bbox={'facecolor':'white', 'alpha':0.5, 'pad':10})

    # 플롯 정리
    # plt.tight_layout(rect=[0, 0.1, 1, 1])  # 여백 조정
    # plt.subplots_adjust(bottom=0.2)  # 하단 여백 조정

    # 이미지 파일로 저장
    final_dir = grad_dir + f"{img_path.name}_gradcam.png"
    plt.imshow(img)
    plt.imshow(cam_resized, alpha=0.6, cmap='jet')
    plt.axis('off')
    plt.savefig(final_dir, bbox_inches='tight', pad_inches=0)

    # 플롯을 닫아 화면에 출력되지 않도록 함
    plt.close()

    return (final_dir, pred_idx.item(), pred_class, probs[pred_idx].item())

def gradcam(learn, x):
    model = learn.model.eval()

    last_conv = find_last_conv(model)

    activations = None
    gradients = None

    def forward_hook(module, input, output):
        nonlocal activations
        activations = output.detach()

    def backward_hook(module, grad_input, grad_output):
        nonlocal gradients
        gradients = grad_output[0].detach()

    forward_handle = last_conv.register_forward_hook(forward_hook)
    backward_handle = last_conv.register_full_backward_hook(backward_hook)

    with torch.set_grad_enabled(True):
        outputs = model(x)
        class_idx = outputs.argmax(dim=1)
        target = outputs[torch.arange(outputs.size(0)), class_idx]
        target.sum().backward()

    forward_handle.remove()
    backward_handle.remove()

    weights = F.adaptive_avg_pool2d(gradients, 1)
    cam = torch.sum(weights * activations, dim=1, keepdim=True)
    cam = F.relu(cam)
    cam = F.interpolate(cam, x.shape[2:], mode='bilinear', align_corners=False)
    cam = cam.squeeze().cpu().numpy()
    cam = (cam - cam.min()) / (cam.max() - cam.min())

    return cam

def find_last_conv(model):
    for module in reversed(list(model.modules())):
        if isinstance(module, nn.Conv2d):
            return module
    raise ValueError("No convolutional layer found in the model")

def apply_gradcam_to_directory(learn, directory_path):
    img_lst = []
    pred_idx_lst = []
    pred_class_lst = []
    probs_lst = []
    # 디렉토리 내의 모든 이미지 파일 가져오기
    image_files = get_image_files(directory_path)
    for i, img_path in enumerate(image_files):
        if i % 5 == 4:
          clear_output(wait=True)
        print(f"Applying GradCAM to: {img_path.name}")
        image_dir, pred_idx, pred_class, probs = apply_gradcam(learn, img_path)
        img_lst.append(image_dir)
        pred_idx_lst.append(pred_idx)
        pred_class_lst.append(pred_class)
        probs_lst.append(probs)

    dataset = {
        "img_path": img_lst,
        "pred_idx": pred_idx_lst,
        "pred_class": pred_class_lst,
        "probs": probs_lst
    }
    return dataset

# GradCAM 적용
dataset = apply_gradcam_to_directory(learn, test_image_path)

Applying GradCAM to: zzybuwyuro.png


Applying GradCAM to: zzykiwwjgm.png


Applying GradCAM to: zzyxaefoft.png


In [9]:
import pandas as pd

# DataFrame 생성
df = pd.DataFrame(dataset)

# question열 생성
for i in range(len(df)):
  prob = round(df.at[i, "probs"], 4)
  if df.at[i, "pred_idx"]==0:
    df.at[i, "question"] = f"주어진 이미지는 AI 생성 이미지와 실제 이미지를 분류한 GradCam이야.현재 분류 결과는 실제 이미지로 분류되었고, 확률은 {prob}이야. 분류 결과와 확률 수치를 참고해서 GradCam 결과를 한 줄로 설명해줬으면 좋겠어."
  else:
    df.at[i, "question"] = f"주어진 이미지는 AI 생성 이미지와 실제 이미지를 분류한 GradCam이야.현재 분류 결과는 AI 이미지로 분류되었고, 확률은 {prob}이야. 분류 결과와 확률 수치를 참고해서 GradCam 결과를 한 줄로 설명해줬으면 좋겠어."

# img_path 기준 정렬
df.sort_values(by='img_path', inplace=True)

# 불필요한 열 드랍
df.drop(columns=['pred_idx','pred_class','probs'], inplace=True)

# CSV 파일로 저장
csv_file_path = dataset_dir + "data.csv"
xlsx_file_path = dataset_dir + "data.xlsx"
df.to_csv(csv_file_path, index=False)  # 인덱스를 제외하고 저장
df.to_excel(xlsx_file_path, index=False)  # 인덱스를 제외하고 저장

In [12]:
!pip install openpyxl

In [14]:
final_csv_path = dataset_dir + "final_data.csv"
final_df = pd.read_excel(xlsx_file_path)
final_df.to_csv(csv_file_path, index=False)